# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770314135791                   -0.53    8.0         
  2   -2.771678867929       -2.86       -1.31    1.0    222ms
  3   -2.771713506291       -4.46       -2.61    1.0    137ms
  4   -2.771714711813       -5.92       -4.05    2.0    160ms
  5   -2.771714715167       -8.47       -4.57    2.0    227ms
  6   -2.771714715245      -10.11       -5.55    1.0    144ms
  7   -2.771714715250      -11.39       -5.95    2.0    166ms
  8   -2.771714715250      -12.94       -6.74    1.0    165ms
  9   -2.771714715250      -14.35       -7.42    2.0    182ms
 10   -2.771714715250      -14.12       -8.32    1.0    183ms


-0.00013457470026206645

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770484893047                   -0.52    9.0         
  2   -2.771778354067       -2.89       -1.32    1.0    137ms
  3   -2.771801585318       -4.63       -2.44    1.0    181ms
  4   -2.771801991457       -6.39       -3.14    1.0    142ms
  5   -2.771802073998       -7.08       -4.06    2.0    170ms
  6   -2.771802074350       -9.45       -4.43    1.0    206ms
  7   -2.771802074471       -9.91       -5.32    1.0    148ms
  8   -2.771802074476      -11.32       -5.83    2.0    178ms
  9   -2.771802074476      -12.91       -6.37    1.0    184ms
 10   -2.771802074476      -13.57       -6.73    1.0    168ms
 11   -2.771802074476   +  -14.21       -7.32    2.0    198ms
 12   -2.771802074476      -13.97       -7.68    1.0    168ms
 13   -2.771802074476      -13.95       -8.43    1.0    167ms


0.01761222087298478

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457470026206645
Displaced dipole:  0.01761222087298478
Polarizability :   1.7746795573246845


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920784742e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551862990e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852776791015e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694603632247e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088789569006e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.298138462531e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.357847301260e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.357847301260e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.072703509952e-10
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.627258228481e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 3.453614569512e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 3.453614569512e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 3.453455103386495e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.